In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# The target URL
url = 'https://qmrawiki.org/node/402'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find elements by HTML tags, attributes, or CSS class
    titles = soup.find_all('h1')
    for title in titles:
        print(title.get_text())
    tables_data = soup.find_all('table')
    
    for table_data in tables_data[3]:
        table_row_data = [row for row in table_data.get_text().split('\n') if row != '']
        if len(table_row_data):
            columns = table_row_data[1:5]
            experiment_data = []
            print(columns)
            i = 5
            while i <= len(table_row_data):
                experiment_data
                row_data = table_row_data[i:i+4]
                i+=4 
                if(len(row_data) > 0):
                    experiment_data.append(row_data)
    df = pd.DataFrame(columns=columns, data=experiment_data)
    print(df)         
else:
    print('Failed to retrieve the webpage')



Optimization Output for Exp. 183
['Dose', 'Infected', 'Non-infected', 'Total']
   Dose Infected Non-infected Total
0   100        2            2     4
1   300        2            3     5
2  1000        1            2     3
3  3000        3            1     4


In [12]:
url_new = 'https://qmrawiki.org/experiments/cryptosporidium-parvum'

response = requests.get(url_new)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    experiments_container = soup.find(class_='view-pathogen-dose-response-experiments')
    
    #experiments = experiments_container.find_all(class_='response-table-inner')

    #for experiment in experiments:
        #print(experiment.get_text().split(' '))
        
    

['\n\n108', '\n\n', '', '', '', '', '', '', '', '', '', '', '', '', '', 'oral', '', '', '', '', '', '', '', '', '', '', '', '\n\n', '', '', '', '', '', '', '', '', '', '', '', '', '', '8', '', '', '', '', '', '', '', '', '', '', '', '\n\n', '', '', '', '', '', '', '', '', '', '', '', '', '', 'Iowa', 'strain', '', '', '', '', '', '', '', '', '', '', '', '\n\n', '', '', '', '', '', '', '', '', '', '', '', '', '', 'oocysts', '', '', '', '', '', '', '', '', '', '', '', '\n\n', '', '', '', '', '', '', '', '', '', '', '', '', '', 'human', '', '', '', '', '', '', '', '', '', '', '', '\n\n', '', '', '', '', '', '', '', '', '', '', '', '', '', 'exponential', '', '', '', '', '', '', '', '', '', '', '', '\n\n\nk', '=', '4.19E-03\nLD50/ID50', '=', '1.65E+02', '\n', '\n\n', '', '', '', '', '', '', '', '', '', '', '', '', '', 'infection', '', '', '', '', '', '', '', '', '', '', '', '\n\nDuPont,', 'H', 'L.,', 'et', 'al.', '"The', 'infectivity', 'of', 'Cryptosporidium', 'parvum', 'in', 'healthy', 'vol

# Step 1, iterating over all the dose response pages and going to next page

In [2]:
dose_responses_url = 'https://qmrawiki.org/framework/dose-response/experiments'
response = requests.get(dose_responses_url)

In [3]:
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    

In [86]:
individual_dose_responses_list = soup.find(class_='quicktabs-views-group')

dose_response_url = individual_dose_responses_list.find_all('span')
base_url = 'https://qmrawiki.org'
for dose_responses in dose_response_url:
    temps = dose_responses.find_all('a')
    for temp in temps:
        experiment_overview_url = temp.get('href')
        extract_from_dose_response_page(base_url+experiment_overview_url, base_url)
        break

/node/1787
Dose response has inline dataset
<tr class="even"><td class="row_2 col_0">100</td><td class="row_2 col_1">0</td><td class="row_2 col_2">16</td><td class="row_2 col_3">16</td> </tr>
<tr class="even"><td class="row_4 col_0">10000</td><td class="row_4 col_1">5</td><td class="row_4 col_2">3</td><td class="row_4 col_3">8</td> </tr>
<td class="row_1 col_0">10</td>
<td class="row_1 col_1">0</td>
<td class="row_1 col_2">8</td>
<td class="row_1 col_3">8</td>
<td class="row_3 col_0">1000</td>
<td class="row_3 col_1">2</td>
<td class="row_3 col_2">16</td>
<td class="row_3 col_3">18</td>
(4, 4)


In [22]:
#TODO: Have to adjust for pages that have the experiment data inside Experiment Dataset
def extract_from_dose_response_page(url, base_url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        experiments_container = soup.select('td.views-field-field-experiment-id-text')
        
        for container in experiments_container:
            experiment_url = container.find('a').get('href')
            print(experiment_url)            
            extract_from_experiment_page(base_url + experiment_url)

In [24]:
def extract_from_experiment_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        experiment_dataset = soup.find_all(id='tablefield-0')

        for data in experiment_dataset:
            experiment_data_table = data.find_all('tr')
            if(len(experiment_data_table) > 0):
                print('Dose response has inline dataset')
                parse_meta_table(experiment_data_table, response.text)
                #TODO: Add function here to extract the data if the dataset is embeded
            else:
                experiment_data_html = extract_html_table(response.text)


In [85]:
def parse_meta_table(data, page_txt):
    i = 0
    soup = BeautifulSoup(page_txt, 'html.parser')
    odds = soup.select('tr.odd')    
    evens = soup.select('tr.even')
    columns = soup.select('th.row_0')
    column_names = [column.get_text() for column in columns]
    rows = []
    even_counter = 2
    odd_counter = 1
    for even in evens:
        
        row = []
        for n in even.find_all(class_=f'row_{even_counter}'):
    
            row.append(n.get_text())
        even_counter += 2
        rows.append(row)
    for odd in odds:
        row = []
        
        for n in odd.find_all(class_=f'row_{odd_counter}'):
            
            row.append(n.get_text())
        rows.append(row)     
        odd_counter += 2
    df = pd.DataFrame(columns=column_names, data=rows) 
    print(df.shape)
    
    
        